# Text Summarization Service

This notebook is meant to demonstrate the transformation of an annotated notebook into a HTTP API using the Jupyter kernel gateway. The result is a simple scotch recommendation engine.

The original scotch data is from [https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html](https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html).

In [1]:
import pandas as pd
import pickle
import requests
import json

## Data

We read the scotch data from a public Box URL to make this notebook more portable. This is acceptable for small, public, demo data which is what we have here.

In [3]:
features_uri = 'https://ibm.box.com/shared/static/2vntdqbozf9lzmukkeoq1lfi2pcb00j1.dataframe' 
sim_uri = 'https://ibm.box.com/shared/static/54kzs5zquv0vjycemjckjbh0n00e7m5t.dataframe'

In [4]:
resp = requests.get(features_uri)
resp.raise_for_status()
features_df = pickle.loads(resp.content)

In [5]:
resp = requests.get(sim_uri)
resp.raise_for_status()
sim_df = pickle.loads(resp.content)

Drop the cluster column. Don't need it here.

In [6]:
features_df = features_df.drop('cluster', axis=1)

## API

We need to define a global `REQUEST` JSON string that will be replaced on each invocation of the API. We only care about path parameters and query string arguments, so we default those to blank here for development.

In [7]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {}
})

Provide a way to get the names of all the scotches known by the model.

In [8]:
# GET /scotches
names = sim_df.columns.tolist()
print(json.dumps(dict(names=names)))

{"names": ["Aberfeldy", "Aberlour", "AnCnoc", "Ardbeg", "Ardmore", "ArranIsleOf", "Auchentoshan", "Auchroisk", "Aultmore", "Balblair", "Balmenach", "Belvenie", "BenNevis", "Benriach", "Benrinnes", "Benromach", "Bladnoch", "BlairAthol", "Bowmore", "Bruichladdich", "Bunnahabhain", "Caol Ila", "Cardhu", "Clynelish", "Craigallechie", "Craigganmore", "Dailuaine", "Dalmore", "Dalwhinnie", "Deanston", "Dufftown", "Edradour", "GlenDeveronMacduff", "GlenElgin", "GlenGarioch", "GlenGrant", "GlenKeith", "GlenMoray", "GlenOrd", "GlenScotia", "GlenSpey", "Glenallachie", "Glendronach", "Glendullan", "Glenfarclas", "Glenfiddich", "Glengoyne", "Glenkinchie", "Glenlivet", "Glenlossie", "Glenmorangie", "Glenrothes", "Glenturret", "Highland Park", "Inchgower", "Isle of Jura", "Knochando", "Lagavulin", "Laphroig", "Linkwood", "Loch Lomond", "Longmorn", "Macallan", "Mannochmore", "Miltonduff", "Mortlach", "Oban", "OldFettercairn", "OldPulteney", "RoyalBrackla", "RoyalLochnagar", "Scapa", "Speyburn", "Speys

In [13]:
# GET /ping
pong = {"ping": "pong"}
print(json.dumps(pong))

{"ping": "pong"}


Let clients query for features about a specific scotch given its name.

In [8]:
# GET /scotches/:scotch
request = json.loads(REQUEST)
name = request['path'].get('scotch', 'Talisker')
features = features_df.loc[name]
# can't use to_dict because it retains numpy types which blow up when we json.dumps
print('{"features":%s}' % features.to_json())

{"features":{"Body":4,"Sweetness":2,"Smoky":3,"Medicinal":3,"Tobacco":0,"Honey":1,"Spicy":3,"Winey":0,"Nutty":1,"Malty":2,"Fruity":2}}


Let clients request a set of scotches similar to the one named. Let clients specify how many results they wish to receive (`count`) and if they want all of the raw feature data included in the result or not (`include_features`).

In [9]:
# GET /scotches/:scotch/similar
request = json.loads(REQUEST)
name = request['path'].get('scotch', 'Talisker')
count = request['args'].get('count', 5)
inc_features = request['args'].get('include_features', True)

similar = sim_df[name].order(ascending=False)
similar.name = 'Similarity'
df = pd.DataFrame(similar).ix[1:count+1]

if inc_features:
    df = df.join(features_df)
    
df = df.reset_index().rename(columns={'Distillery': 'Name'})
result = {
    'recommendations' : [row[1].to_dict() for row in df.iterrows()],
    'for': name
}
print(json.dumps(result))

{"recommendations": [{"Name": "Clynelish", "Sweetness": 2, "Medicinal": 3, "Smoky": 3, "Malty": 1, "Body": 3, "Tobacco": 1, "Fruity": 2, "Honey": 0, "Similarity": 0.9605849651931131, "Spicy": 2, "Nutty": 1, "Winey": 0}, {"Name": "Ardbeg", "Sweetness": 1, "Medicinal": 4, "Smoky": 4, "Malty": 2, "Body": 4, "Tobacco": 0, "Fruity": 1, "Honey": 0, "Similarity": 0.9484168381883901, "Spicy": 2, "Nutty": 1, "Winey": 0}, {"Name": "Caol Ila", "Sweetness": 1, "Medicinal": 2, "Smoky": 4, "Malty": 1, "Body": 3, "Tobacco": 1, "Fruity": 1, "Honey": 0, "Similarity": 0.8992710312446166, "Spicy": 2, "Nutty": 2, "Winey": 0}, {"Name": "Lagavulin", "Sweetness": 1, "Medicinal": 4, "Smoky": 4, "Malty": 1, "Body": 4, "Tobacco": 1, "Fruity": 1, "Honey": 0, "Similarity": 0.8695980950674561, "Spicy": 1, "Nutty": 1, "Winey": 2}, {"Name": "Dalmore", "Sweetness": 2, "Medicinal": 1, "Smoky": 2, "Malty": 2, "Body": 3, "Tobacco": 0, "Fruity": 3, "Honey": 1, "Similarity": 0.839754059379345, "Spicy": 2, "Nutty": 1, "Win

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: order is deprecated, use sort_values(...)
